In [1]:
import re
import json
from glob import glob
from tqdm import tqdm
import random
from unidecode import unidecode

In [2]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions'
]

In [3]:
files = glob('train_*.jsonl.translated')
files

['train_0.jsonl.translated',
 'train_7.jsonl.translated',
 'train_4.jsonl.translated',
 'train_9.jsonl.translated',
 'train_6.jsonl.translated',
 'train_1.jsonl.translated',
 'train_8.jsonl.translated',
 'train_3.jsonl.translated',
 'train_5.jsonl.translated']

In [4]:
# for f in files:
#     with open(f) as fopen:
#         for l in fopen:
#             l = json.loads(l)
#             for l_ in l:
#                 if 'maklumat umum' in l_['value_ms']:
#                     print(l_)
                    

In [11]:
for f in files:

    all_data = []
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            all_data.append(l)
    
    all_texts = []
    for i in tqdm(range(len(all_data))):
        texts = []
        for no, d in enumerate(all_data[i]):

            if no % 2 == 0:
                t = '<manusia>: '
                if random.random() > 0.5:
                    c = 'value'
                else:
                    c = 'value_ms'
            else:
                c = 'value_ms'
                t = '<bot>: '

            if d[c] is None:
                break
                
            if no < (len(all_data[i]) - 1):
                next_text = all_data[i][no + 1].get('value') or ''
                next_text_ms = all_data[i][no + 1].get('value_ms') or ''
            else:
                next_text = ''
                next_text_ms = ''

            current_text = d.get('value') or ''
            current_text_ms = d.get('value_ms') or ''
            previous_text = all_data[i][no - 1].get('value') or ''
            previous_text_ms = all_data[i][no - 1].get('value_ms') or ''

            # remove alignments    
            if no % 2 == 0 and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                break
            if no % 2 != 0 and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                break
                
            # remove alignments    
            if no % 2 == 0 and (any([b in current_text_ms.lower() for b in break_at]) or any([b in next_text_ms.lower() for b in break_at])):
                # print(current_text_ms, break_at, [b for b in break_at if b in current_text_ms.lower()])
                break
            if no % 2 != 0 and (any([b in current_text_ms.lower() for b in break_at]) or any([b in previous_text_ms.lower() for b in break_at])):
                break

            n = d[c]
            n = n.strip()
            if len(n) < 3:
                break
                
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
                
#             if len(hypens):
#                 print(hypens, n)
                
            t = t + n
            texts.append(t)

        if len(texts) % 2 != 0:
            texts = texts[:-1]
        if not len(texts):
            continue
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue
        all_texts.append('\n'.join(texts))
    
    with open(f'{f}.malay.no-alignment', 'w') as fopen:
        for t in all_texts:
            fopen.write(f'{json.dumps(t)}\n')

100%|██████████████████████████████████| 150000/150000 [02:39<00:00, 941.15it/s]


In [13]:
from huggingface_hub import HfApi

api = HfApi()

In [ ]:
for f in glob('train_*.jsonl.translated.malay.no-alignment'):
    print(f)
    
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/google-translate-ultrachat',
        repo_type='dataset',
    )

train_6.jsonl.translated.malay.no-alignment


train_6.jsonl.translated.malay.no-alignment:   0%|          | 0.00/824M [00:00<?, ?B/s]